# Create an empty project in Neo4j desktop with an empty DB in the project, then start it.
## Add the AAPOC and GDS plugins in your desktop Neo4j project.

## Steps 1-4 | be sure to change PORT for URI and PASSWORD for AUTH!

In [3]:
import pandas as pd

In [4]:
from neo4j import GraphDatabase

In [5]:
NEO4J_URI = "bolt://localhost:7687"

In [6]:
NEO4J_AUTH = ( "neo4j",  "Penguin23")

In [7]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

## Step 5: SHARED_PII relationship

In [8]:
query = """
MATCH ( c:Client )-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ] -> (n) <- [ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]- ( d:Client )
WHERE id(c) < id(d)
WITH c, d, count(*) AS cnt
MERGE (c) - [ :SHARED_PII { count: cnt } ] -> (d)
"""

In [9]:
driver.execute_query(query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)} {position: line: 2, column: 45, offset: 45} for query: '\nMATCH ( c:Client )-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ] -> (n) <- [ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]- ( d:Client )\nWHERE id(c) < id(d)\nWITH c, d, count(*) AS cnt\nMERGE (c) - [ :SHARED_PII { count: cnt } ] -> (d)\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationshi

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x00000261B6C38CB0>, keys=[])

## Step 6: clientClusters projection


In [10]:
query2 = """
CALL gds.graph.project('clientClusters' , { 
    Client: {
        label: 'Client' 
    }
}, { 
    SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
            count: {
                property: 'count' 
            }
        }
    }
}
)
YIELD graphName, nodeCount, relationshipCount
"""


In [11]:
driver.execute_query(query2)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'clientClusters' already exists.}

## Step 7: Identify clusters

In [48]:
def run_wcc_query(driver):
    query3 = """
    CALL gds.wcc.stream(
      'clientClusters', 
      {
        nodeLabels: ['Client'], 
        relationshipTypes: ['SHARED_PII'], 
        consecutiveIds: true
      }
    )
    YIELD nodeId, componentId 
    RETURN gds.util.asNode(nodeId).id AS clientId,
    componentId AS clusterId"""

    with driver.session() as session:
        result = session.run(query3)

        records = result.data() 
        df = pd.DataFrame(records)

    return df
    

In [49]:
pd_df = run_wcc_query(driver)

In [50]:
pd.set_option('display.max_columns', 2)

In [51]:
pd_df

,clientId,clusterId
0,4997933060327094,0
1,4776276949898423,1
2,4858607188760216,2
3,4287186486553145,3
4,4661202154682409,4
...,...,...
2428,4413385955087620,1767
2429,4550448544478545,1862
2430,4114683318919154,334
2431,4172817689754167,2113


## Step 8: Marking Clients

In [52]:

query4 = """
    CALL gds.wcc.stream(
      'clientClusters', 
      {
        nodeLabels: ['Client'], 
        relationshipTypes: ['SHARED_PII'], 
        consecutiveIds: true
      }
    )
    YIELD nodeId, componentId
    WITH gds.util.asNode(nodeId) AS client, componentId AS clusterId
    WITH clusterId, collect(client.id) AS clients
    WITH clusterId, clients, size(clients) AS clusterSize 
    WHERE clusterSize >= 2
    UNWIND clients AS client
    MATCH (c:Client {id: client})  // Assuming 'id' property here matches
    SET c.secondPartyFraudRing = clusterId
    """


In [53]:
driver.execute_query(query4)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000002057DAB2A50>, keys=[])

## Step 9: Bipartite Graph

In [54]:
query5 = """
// first, find clients
MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
WITH collect(c) as clients

// second, find the PII nodes
MATCH (n) WHERE n:Email OR n:Phone OR n:SSN

// combine the two sets of nodes
WITH clients, collect(n) AS piis
WITH clients + piis AS nodes

// use only the clients that belong to a cluster of size >= 2
// as per STEP 8
MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing is NOT NULL

// now make the bipartite graph,
// with a new relationship named HAS_PII
WITH nodes, collect({source: c, target: p}) as relationships

// use a Cypher projection
// not the usual native projection
CALL gds.graph.project.cypher(
'similarity',
"UNWIND $nodes as n
RETURN id(n) AS id,labels(n) AS labels",
"UNWIND $relationships as r
RETURN id(r['source']) AS source, id(r['target']) AS target,
'HAS_PII' as type",
{ parameters:
{ nodes: nodes,
relationships: relationships }
}
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
"""

In [55]:
driver.execute_query(query5)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)} {position: line: 15, column: 44, offset: 403} for query: '\n// first, find clients\nMATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL\nWITH collect(c) as clients\n\n// second, find the PII nodes\nMATCH (n) WHERE n:Email OR n:Phone OR n:SSN\n\n// combine the two sets of nodes\nWITH clients, collect(n) AS piis\nWITH clients + piis AS nodes\n\n// use only the clients that belong to a cluster of size >= 2\n// as per STEP 8\nMATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)\nWHERE c.secondPartyFraudRing is NOT NULL\n\n// now make the bipartite graph,\n// with a new relatio

EagerResult(records=[<Record graphName='similarity' nodeCount=7037 relationshipCount=1008>], summary=<neo4j._work.summary.ResultSummary object at 0x000002057DAB1C40>, keys=['graphName', 'nodeCount', 'relationshipCount'])

## Step 10: Creating the SIMILAR_TO Relationship

In [56]:
step10 = """
CALL gds.nodeSimilarity.mutate(
  'similarity',
  {
      mutateProperty: 'jaccardScore',
      mutateRelationshipType: 'SIMILAR_TO',
      topK: 15
  }
)
YIELD
  preProcessingMillis,
  computeMillis,
  mutateMillis,
  postProcessingMillis,
  relationshipsWritten,
  nodesCompared,
  similarityDistribution,
  configuration
"""

In [57]:
driver.execute_query(step10)

EagerResult(records=[<Record preProcessingMillis=0 computeMillis=110 mutateMillis=173 postProcessingMillis=0 relationshipsWritten=1518 nodesCompared=336 similarityDistribution={'min': 0.19999980926513672, 'p5': 0.19999980926513672, 'max': 1.000007629394531, 'p99': 1.0000066757202148, 'p1': 0.19999980926513672, 'p10': 0.19999980926513672, 'p90': 0.5000028610229492, 'p50': 0.19999980926513672, 'p25': 0.19999980926513672, 'p75': 0.5000028610229492, 'p95': 1.0000066757202148, 'mean': 0.3347832789062983, 'p100': 1.0000066757202148, 'stdDev': 0.20693545125522303} configuration={'mutateProperty': 'jaccardScore', 'jobId': 'a74ba808-7748-469a-a56b-8fd4de6ef87b', 'topN': 0, 'upperDegreeCutoff': 2147483647, 'topK': 15, 'similarityCutoff': 1e-42, 'sudo': False, 'degreeCutoff': 1, 'useComponents': 'false', 'mutateRelationshipType': 'SIMILAR_TO', 'bottomN': 0, 'bottomK': 10, 'logProgress': True, 'nodeLabels': ['*'], 'concurrency': 4, 'relationshipTypes': ['*'], 'similarityMetric': 'JACCARD'}>], summ

# Step 11


In [1]:
step11 = """
CALL gds.graph.writeRelationship(
    'similarity',           
    'SIMILAR_TO',          
    'jaccardScore'       
)
YIELD
    writeMillis,
    graphName, 
    relationshipType,
    relationshipsWritten,
    relationshipProperty,
    propertiesWritten;
"""

In [2]:
driver.execute_query(step11)

NameError: name 'driver' is not defined

# Step 12

In [58]:
step12 = """
CALL gds.degree.write(
    'similarity',
    {
        nodeLabels: ['Client'],
        relationshipTypes: ['SIMILAR_TO'],
        relationshipWeightProperty: 'jaccardScore',
        writeProperty: 'secondPartyFraudScore'
    }
)
YIELD
    nodePropertiesWritten,
    centralityDistribution,
    configuration
"""

In [59]:
driver.execute_query(step12)

EagerResult(records=[<Record nodePropertiesWritten=336 centralityDistribution={'min': 0.19999980926513672, 'max': 3.500015258789062, 'p90': 2.500014305114746, 'p999': 3.500014305114746, 'p99': 3.1000051498413086, 'p50': 1.4000005722045898, 'p75': 2.000014305114746, 'p95': 2.800002098083496, 'mean': 1.5125004393713815} configuration={'writeProperty': 'secondPartyFraudScore', 'orientation': 'NATURAL', 'jobId': 'fd4bcb92-caad-46a2-8b52-a6455aa87ac2', 'logProgress': True, 'relationshipWeightProperty': 'jaccardScore', 'nodeLabels': ['Client'], 'relationshipTypes': ['SIMILAR_TO'], 'concurrency': 4, 'sudo': False, 'writeToResultStore': False, 'writeConcurrency': 4}>], summary=<neo4j._work.summary.ResultSummary object at 0x000002057DBB5070>, keys=['nodePropertiesWritten', 'centralityDistribution', 'configuration'])

# Step 13

In [60]:
step13 = """
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
MATCH (c:Client)
WHERE c.secondPartyFraudScore > threshold
SET c:SecondPartyFraudster
SET c.isSecondPartyFraudster = true
RETURN count(c) as fraudsters, threshold
"""

In [61]:
driver.execute_query(step13)

EagerResult(records=[<Record fraudsters=17 threshold=2.7250000000000005>], summary=<neo4j._work.summary.ResultSummary object at 0x000002057DAB1340>, keys=['fraudsters', 'threshold'])

# Step 14

In [69]:
def run_14(driver):
    step14 = """
    MATCH (c:Client)
    WHERE c:SecondPartyFraudster
    RETURN c.name as client_name, c.id as client_id
    """
    with driver.session() as session:
        result = session.run(step14)

        records = result.data() 
        df = pd.DataFrame(records)

    return df

pd_df = run_14(driver)
pd_df

,client_name,client_id
0,Brooklyn Harrison,4029043591201321
1,Aiden Hurst,4583937317122539
2,Sarah Klein,4912587051525728
3,Kennedy Keith,4446118457512030
4,Samantha Mueller,4717521340444448
5,Benjamin Moss,4189330002136246
6,Charlotte Foster,4024985944102082
7,Ryan Patel,4614177132519923
8,Landon Welch,4359490519123048
9,Allison Alvarez,4371660075922934
